In [1]:
import sklearn
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Data/CleanedData.csv')

In [3]:
df.head()

,subject,transformed text
0,1,donald trump wish american happi new year leav...
1,1,hous intellig committe chairman devin nune go ...
2,1,friday reveal former milwauke sheriff david cl...
3,1,christma day donald trump announc would back w...
4,1,pope franci use annual christma day messag reb...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=2000)

In [5]:
x= cv.fit_transform(df['transformed text']).toarray()
y= df['subject'].values

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state = 42)

In [7]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [8]:
model_params= {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C': [5, 10],
            'kernel': ['poly', 'rbf']
        }
    },
    'random_forest':{
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators': range(1,2),
            'criterion': ['entropy', 'log_loss']
        }
    },
    'logistic_regression':{
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params':{
            'C': [5, 10],
            'penalty': ['l1'],
        }
    },
    'gaussian_nb':{
        'model': GaussianNB(priors=None, var_smoothing=1e-09),
        'params':{
            # No hyperparameters to tune for this model type yet...
        }
    },
    'multinomial_nb':{
        'model': MultinomialNB(alpha=1),
        'params':{
            #'alpha' : np.linspace(.01,.99, num = 3)
        }
    },
    'bernoulli_nb':{
        'model': BernoulliNB(alpha=1, binarize=0.0, fit_prior=True, class_prior=None),
        'params':{
            #'alpha' : np.linspace(.01,.99, num = 3)
        }
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(random_state= 0),
        'params':{
            'criterion': ['entropy'],
            'max_depth':[2,3]
        }
    }
}

In [9]:
from sklearn.model_selection import GridSearchCV
import time

In [10]:
scores= []
start = time.time()
for model_name, mp in model_params.items():
    clf= GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    print("1 epoch Done")

end = time.time()
print(start)
print(end)
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

1 epoch Done


/home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/joblib/parallel.py", line 1863, in __call__
    return output if self.retu

1 epoch Done
1 epoch Done
1 epoch Done
1 epoch Done
1 epoch Done
1 epoch Done
1694340395.7548141
1694352462.054944
The time of execution of above program is : 12066300.129890442 ms


In [11]:
df= pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.800723,"{'C': 5, 'kernel': 'rbf'}"
1,random_forest,0.614164,"{'criterion': 'entropy', 'n_estimators': 1}"
2,logistic_regression,0.769331,"{'C': 5, 'penalty': 'l1'}"
3,gaussian_nb,0.557161,{}
4,multinomial_nb,0.693626,{}
5,bernoulli_nb,0.702818,{}
6,decision_tree,0.741986,"{'criterion': 'entropy', 'max_depth': 3}"
